In [1]:
import numpy as np
import pandas as pd
import scanpy as sc
import hotspot


sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80, facecolor='white')

adata = sc.read_10x_mtx(
    'data/filtered_gene_bc_matrices/hg19/',  # the directory with the `.mtx` file
    var_names='gene_symbols',                # use gene symbols for the variable names (variables-axis index)
    cache=True)                              # write a cache file for faster subsequent reading

adata.var_names_make_unique()
adata

scanpy==1.9.1 anndata==0.8.0 umap==0.5.3 numpy==1.23.5 scipy==1.10.0 pandas==1.5.3 scikit-learn==1.2.1 statsmodels==0.13.5 python-igraph==0.10.4 pynndescent==0.5.8
... reading from cache file cache\data-filtered_gene_bc_matrices-hg19-matrix.h5ad


AnnData object with n_obs × n_vars = 2700 × 32738
    var: 'gene_ids'

In [2]:
adata.layers["counts"] = adata.X.copy()
sc.pp.filter_genes(adata, min_cells=3)

filtered out 19024 genes that are detected in less than 3 cells


In [3]:
adata.var['mt'] = adata.var_names.str.startswith('MT-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)

In [4]:
adata = adata[adata.obs.n_genes_by_counts > 200, :]
adata = adata[adata.obs.n_genes_by_counts < 2500, :]
adata = adata[adata.obs.pct_counts_mt < 5, :]

In [5]:
# Pre-processing
sc.pp.filter_genes(adata, min_cells=3)
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)

# sc.pp.highly_variable_genes(adata, min_mean=0.1, max_mean=10, min_disp=0.1) 
adata.raw = adata

# adata = adata[:, adata.var.highly_variable]

sc.pp.regress_out(adata, ['total_counts', 'pct_counts_mt'])
sc.pp.scale(adata, max_value=10)

#PCA 
sc.tl.pca(adata, svd_solver='arpack')

filtered out 58 genes that are detected in less than 3 cells
normalizing counts per cell
    finished (0:00:00)
regressing out ['total_counts', 'pct_counts_mt']
    sparse input is densified and may lead to high memory use


C:\Users\denni\anaconda3\envs\bioinformatics\lib\site-packages\scanpy\preprocessing\_simple.py:251: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  adata.var['n_cells'] = number


    finished (0:00:39)
computing PCA
    with n_comps=50
    finished (0:00:03)


In [6]:
hs = hotspot.Hotspot(
    adata,
    layer_key="counts",
    model='danb',
    latent_obsm_key="X_pca",
    umi_counts_obs_key="total_counts"
)

C:\Users\denni\anaconda3\envs\bioinformatics\lib\site-packages\hotspot\hotspot.py:98: UserWarning: Hotspot will work faster when counts are a csc sparse matrix.
  warnings.warn(


In [7]:
hs.create_knn_graph(weighted_graph=False, n_neighbors=30)

In [8]:
hs_results = hs.compute_autocorrelations() 

100%|███████████████████████████████████████████████████████████████████████████| 13656/13656 [00:36<00:00, 370.20it/s]


In [9]:
hs_results.C

Gene
PPBP       0.436694
CLU        0.346361
GNG11      0.427145
S100A8     0.633780
CD9        0.214529
             ...   
ARL2      -0.001874
ILF3      -0.002154
ANAPC13   -0.001664
MTIF3     -0.003475
TTC1      -0.001589
Name: C, Length: 13656, dtype: float64

In [10]:
hs_genes = hs_results.loc[hs_results.FDR < 0.05].index # Select genes

local_correlations = hs.compute_local_correlations(hs_genes) # jobs for parallelization

Computing pair-wise local correlation on 4747 features...


100%|██████████████████████████████████████████████████████████████████| 11264631/11264631 [1:32:24<00:00, 2031.82it/s]


In [11]:
modules = hs.create_modules(
    min_gene_threshold=30, core_only=True, fdr_threshold=0.05
)

In [12]:
module_scores = hs.calculate_module_scores()

Computing scores for 19 modules...


100%|██████████████████████████████████████████████████████████████████████████████████| 19/19 [00:01<00:00, 13.75it/s]


In [13]:
hs.module_scores

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
AAACATACAACCAC-1,-0.727609,2.624979,-3.311350,0.805688,-2.160814,-1.756093,1.608772,-1.186783,-0.802973,-0.644605,0.947214,-0.284562,-0.420781,0.634126,-0.317481,-0.468689,1.477604,0.194645,0.101362
AAACATTGAGCTAC-1,-0.547002,2.090939,-3.155074,-2.245882,9.666570,-1.402685,-2.015072,-1.462063,-0.869494,-0.351993,-1.189009,3.222033,-0.508973,0.394285,0.417333,-0.521530,-0.729691,-0.202885,0.054668
AAACATTGATCAGC-1,-0.514758,0.248048,-3.028685,-0.077924,-1.931712,-1.580731,2.450792,0.100175,-0.419503,-0.633773,1.959789,-0.297110,-0.173899,0.940622,-0.349555,0.091186,0.296206,0.647022,2.308476
AAACCGTGCTTCCG-1,-0.203761,-5.861107,9.287175,-2.624157,-1.257042,4.710341,-2.124619,2.417568,1.330783,1.287826,-1.376093,-0.586650,0.730636,-2.414737,0.251063,0.029899,-0.814138,-1.026194,-0.827805
AAACCGTGTATGCG-1,-0.451362,-6.649067,-0.330066,13.408894,-1.477985,0.463921,-0.655916,5.663859,-0.195100,0.399130,-0.242442,-0.226311,-0.105401,1.087833,-0.088406,0.005410,-0.141315,0.730497,0.297812
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTCGAACTCTCAT-1,0.267775,-4.808762,10.059229,-2.683026,-1.406319,1.889484,-2.239332,1.447883,3.594074,0.129907,-1.271995,-0.216682,0.880483,-2.333885,0.940513,0.867038,-0.733483,-0.967269,-0.991798
TTTCTACTGAGGCA-1,-0.657981,1.183203,-3.023033,-1.448591,6.751172,-1.256827,-1.385403,-0.938995,-0.519361,-0.168586,-0.760116,4.818460,-0.301900,0.755246,0.274111,-0.205369,-0.327468,-0.039595,1.222869
TTTCTACTTCCTCG-1,-0.705997,1.705758,-3.204609,-2.278934,12.361921,-1.263154,-1.717041,-1.872091,-0.849086,-0.358258,-1.072594,0.409305,-0.221099,0.611751,-0.117688,-0.509048,-0.678666,-0.267697,-0.170317
TTTGCATGAGAGGC-1,-0.627192,0.345789,-2.462585,-1.872565,10.973145,-0.691493,-1.680086,-1.436423,-0.626005,0.076245,-1.093764,0.775826,-0.212772,1.197115,0.143104,-0.496551,-0.658349,-0.205316,0.111720
